We are making mulit output non liear nn as shown in image 02.0.png
We will use transfer learning ...conv layers of resnet and we will make ann layer by ourself ..and branching will be done by ourself 


In [ ]:
See face of a person predict his/her age and gender 


In [ ]:
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors




  0%|          | 0.00/331M [00:00<?, ?B/s]
  0%|          | 1.00M/331M [00:01<09:37, 600kB/s]
  1%|          | 2.00M/331M [00:01<04:45, 1.21MB/s]
  1%|          | 3.00M/331M [00:02<02:57, 1.94MB/s]
  1%|          | 4.00M/331M [00:02<02:00, 2.84MB/s]
  2%|▏         | 6.00M/331M [00:02<01:11, 4.75MB/s]
  2%|▏         | 8.00M/331M [00:02<00:52, 6.40MB/s]
  3%|▎         | 10.0M/331M [00:02<00:43, 7.73MB/s]
  4%|▎         | 12.0M/331M [00:02<00:38, 8.75MB/s]
  4%|▍         | 14.0M/331M [00:03<00:34, 9.56MB/s]
  5%|▍         | 16.0M/331M [00:03<00:32, 10.2MB/s]
  5%|▌         | 18.0M/331M [00:03<00:30, 10.6MB/s]
  6%|▌         | 20.0M/331M [00:03<00:31, 10.3MB/s]
  7%|▋         | 22.0M/331M [00:03<00:30, 10.7MB/s]
  7%|▋         | 24.0M/331M [00:04<00:29, 11.1MB/s]
  8%|▊         | 26.0M/331M [00:04<00:28, 11.3MB/s]
  8%|▊         | 28.0M/331M [00:04<00:29, 10.8MB/s]
  9%|▉         | 30.0M/331M [00:04<00:28, 11.3MB/s]
 10%|▉         | 32.0M/331M [00:04<00:27, 11.6MB/s]
 10%|█         | 34.0

In [ ]:
import zipfile
zip = zipfile.ZipFile("/content/utkface-new.zip",'r')
zip.extractall("/content")
zip.close()

In [ ]:
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
folder_path = '/content/utkface_aligned_cropped/UTKFace'

In [3]:
# This is being done to seperate data into 3 columns age , gender , imp_path so it can be easy to extract data and give as input 
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file) 

NameError: name 'os' is not defined

In [ ]:
df = pd.DataFrame({'age':age,'gender':gender,'img':img_path})

In [ ]:
train_df = df.sample(frac=1,random_state=0).iloc[:20000]
test_df = df.sample(frac=1,random_state=0).iloc[20000:]

In [ ]:
# Data augmentation 
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Generator 
train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                    class_mode='multi_output')

test_generator = test_datagen.flow_from_dataframe(test_df,
                                                    directory=folder_path,
                                                    x_col='img',
                                                    y_col=['age','gender'],
                                                    target_size=(200,200),
                                                  class_mode='multi_output')

Main code of making model starts from here


In [4]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [ ]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3))

In [ ]:
resnet = ResNet50(include_top=False, input_shape=(200,200,3)) # include conv layers 

resnet.trainable=False

output = resnet.layers[-1].output 

flatten = Flatten()(output)

# ANN layers 

dense1 = Dense(512, activation='relu')(flatten)
dense2 = Dense(512,activation='relu')(flatten)

dense3 = Dense(512,activation='relu')(dense1)
dense4 = Dense(512,activation='relu')(dense2)

output1 = Dense(1,activation='linear',name='age')(dense3)
output2 = Dense(1,activation='sigmoid',name='gender')(dense4)

In [ ]:
model = Model(inputs=resnet.input,outputs=[output1,output2])

In [ ]:
model.compile(optimizer='adam', loss={'age': 'mae', 'gender': 'binary_crossentropy'}, metrics={'age': 'mae', 'gender': 'accuracy'},loss_weights={'age':1,'gender':99})